In [1]:
cd /home/ubuntu/DistMEC/

/home/ubuntu/DistMEC


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from classes.User import *
from classes.solver import *

In [3]:
def gen_eq_locs(space_1d, nums, offset = 0.5):
    # Generate well spread out locations in square space
    num_across = int(np.floor(np.sqrt(nums)))
    locs = []
    
    inc = space_1d/nums
    
    for i,j in itertools.product(range(num_across), range(num_across)):
        locs += [(i*inc+offset, j*inc+offset)]
    
    return locs

def 

In [4]:
# System Parameters
T = 10000
num_users = 10
num_svrs = 16
num_locs = 16
mu = np.random.uniform(low=0, high = 1, size = [num_users, num_svrs])
w = np.random.uniform(low=0, high=1, size= [num_users, num_svrs])

# Generate visitng locations and server locations
space_1d_dim = 16
usr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_locs)
svr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_svrs)

# Create Users
Users = []
for i in range(num_users):
    Users += [User(usr_locs,svr_locs,w[i])]

# Recorded Values
rewards_record = np.zeros([num_users,num_svrs])
pulls_record = np.zeros([num_users,num_svrs])
ucb = np.zeros([num_users,num_svrs])
regret = np.zeros(T)

In [ ]:
# Initial Stages -- TS = servers, rotating 
arms = list(range(num_users)) 
# w = np.random.uniform(low=0, high=1, size= [num_users, num_svrs])
# optimal = offline_optimal_action(w, mu)

for i in range(num_svrs):
    w = np.random.uniform(low=0, high=1, size= [num_users, num_svrs])
    optimal = offline_optimal_action(w, mu)
    rewards_record, pulls_record = pull_super_arm(arms, mu, rewards_record, pulls_record)
    regret[i] = optimal[1] - expected_reward(arms, mu, w)
    arms = sweep_init_next(arms, num_svrs)
    
for t in range(i+1, T):
    w = np.random.uniform(low=0, high=1, size= [num_users, num_svrs])
    optimal = offline_optimal_action(w, mu)
    ucb = update_ucb(rewards_record, pulls_record, ucb, i, 1)
    arms = offline_optimal_action(w, ucb)[0]
    rewards_record, pulls_record = pull_super_arm(arms, mu, rewards_record, pulls_record)
    regret[t] = optimal[1] - expected_reward(arms, mu, w)